In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("/kaggle/input/child-mind-institute-problematic-internet-use/train.csv")
test = pd.read_csv("/kaggle/input/child-mind-institute-problematic-internet-use/test.csv")
dictionary = pd.read_csv("/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv")
# sample_submission = pd.read_csv("/kaggle/input/dataset/sample_submission.csv")

train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
id_to_test = np.array(test['id'])
time_series = pd.read_csv("/kaggle/input/time-series-features/ts2feature.csv")
train = train.merge(time_series, on = 'id', how='left')
test = test.merge(time_series, on = 'id', how='left')
train 

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,sedentary_por,light_por,moderate_por
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,55.0,NaN,NaN,NaN,Fall,3.0,2.0,NaN,NaN,NaN
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,Fall,46.0,64.0,Summer,0.0,0.0,NaN,NaN,NaN
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,28.0,Fall,38.0,54.0,Summer,2.0,0.0,NaN,NaN,NaN
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,44.0,Summer,31.0,45.0,Winter,0.0,1.0,0.311401,0.583106,0.105493
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,32.0,Winter,35.0,50.0,Fall,1.0,1.0,NaN,NaN,NaN
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,Winter,0.0,NaN,NaN,NaN,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,31.0,Winter,56.0,77.0,Fall,0.0,1.0,0.534781,0.399045,0.066173
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,19.0,Spring,33.0,47.0,Spring,1.0,0.0,0.562736,0.363218,0.074046


In [4]:
from sklearn import preprocessing

encoding = preprocessing.LabelEncoder()

def convert(dataset):
    check = False
    if 'id' in dataset.columns:
        getId = dataset['id']
        dataset = dataset.drop(columns = 'id')
        check = True
    get_columns = dataset.select_dtypes(exclude="number").columns
    for cols in get_columns:
        dataset[cols] = encoding.fit_transform(dataset[cols])
    if check == True:
        return getId, dataset
    return dataset
def reduction(dataset):
    for cols in dataset.columns:
        if (cols != 'sii') and (dataset[cols].isna().sum() > len(dataset[cols]) * 8 / 10):
            dataset.drop(cols, axis=1, errors="ignore", inplace=True)
    return dataset
def fillCategory(dataset):
    cols = dataset.select_dtypes("object").columns
    dataset[cols] = dataset[cols].fillna("Missing")
    dataset[cols] = dataset[cols].astype("category")
    return dataset

In [7]:
common_columns = train.columns.intersection(test.columns)
final_train = pd.DataFrame(train[common_columns])
final_train['sii'] = train['sii'].reset_index(drop=True)

id, final_train = convert(final_train)
final_train = final_train.dropna(subset='sii')
final_train = reduction(final_train)
len(final_train.columns)

60

In [8]:
final_train = fillCategory(final_train)
final_train.isna().sum()

Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                                  0
CGAS-CGAS_Score                            394
Physical-Season                              0
Physical-BMI                               209
Physical-Height                            206
Physical-Weight                            164
Physical-Diastolic_BP                      258
Physical-HeartRate                         250
Physical-Systolic_BP                       258
Fitness_Endurance-Season                     0
Fitness_Endurance-Max_Stage               2005
Fitness_Endurance-Time_Mins               2008
Fitness_Endurance-Time_Sec                2008
FGC-Season                                   0
FGC-FGC_CU                                 817
FGC-FGC_CU_Zone                            852
FGC-FGC_GSND                              1864
FGC-FGC_GSND_Zone                         1872
FGC-FGC_GSD  

In [9]:
new_common_columns = test.columns.intersection(final_train.columns)
final_test = pd.DataFrame(test[new_common_columns])
final_test = convert(final_test)
len(final_test.columns)

59

In [10]:
from sklearn.model_selection import train_test_split
feature_train = final_train.drop(columns = 'sii')
X = np.array(feature_train)
y = np.array(final_train["sii"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

X_to_test = np.array(final_test)
y_to_test = y[:20]


In [11]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
param_distributions = {
    'iterations': [1, 5, 10, 50, 100, 200],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5, 0.8],
    'depth': [1, 2, 3, 5, 10, 12],
}
classifier = CatBoostClassifier(verbose=0)

CatBoost = RandomizedSearchCV(
    estimator=classifier,
    param_distributions=param_distributions,
    n_iter=100, 
    cv=3,
    scoring='f1_macro', 
    random_state=42,
    n_jobs=-1
)

CatBoost.fit(X_train, y_train, use_best_model=False)
prediction1 = CatBoost.predict(X_to_test)

In [ ]:
prediction1 = prediction1.flatten()
prediction1 = pd.Series(prediction1)
submission = pd.DataFrame({
    'id': id_to_test,
    'sii': prediction1
})
submission


In [18]:
submission.to_csv("submission.csv", index=False)